### Univariate CNN Models
1. Data Preparation
2. CNN Model

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time step
n_steps = 3
X, y = split_sequence(raw_seq, n_steps)
print("Actual Input")
print(raw_seq)
print("Reshaped Input")
print(X)
print(X.shape)
print("Desried Output")
print(y)
print(y.shape)

Actual Input
[10, 20, 30, 40, 50, 60, 70, 80, 90]
Reshaped Input
[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]
(6, 3)
Desried Output
[40 50 60 70 80 90]
(6,)


In [3]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)

(6, 3, 1)


In [4]:
# define model
model = Sequential()
model.add(Conv1D(
    filters=64, kernel_size=2, activation='relu',
    input_shape=(n_steps, n_features)
))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')

In [5]:
# fit model
model.fit(x=X, y=y, epochs=1000, verbose=False)

In [6]:
# demo prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

1/1 [==============================] - 0s 57ms/sample
[[101.37697]]
(1, 1)


### Multivariate CNN Models
1. Single step prediction
2. Multi step prediction

In [7]:
# multivariate data preparation
from numpy import array
from numpy import hstack
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))                 
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns                
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
print(dataset.shape)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
(9, 3)


In [8]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [9]:
# define model
models = Sequential()
models.add(Conv1D(
    filters=64, kernel_size=2, activation='relu',
    input_shape=(n_steps, n_features)
))
models.add(MaxPooling1D())
models.add(Flatten())
models.add(Dense(units=50, activation='relu'))
models.add(Dense(units=1))
models.compile(optimizer='adam', loss='mse')
# fit model
models.fit(x=X, y=y, epochs=1000, verbose=False)

In [10]:
# demo prediction
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = models.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[ 80  85]
  [ 90  95]
  [100 105]]]
(1, 3, 2)
1/1 [==============================] - 0s 37ms/sample
[[208.80515]]
(1, 1)


### Multivariate multistep CNN models

In [11]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1)) 
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
print(dataset.shape)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
(9, 3)


In [12]:
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [13]:
# define model
models = Sequential()
models.add(Conv1D(
    filters=64, kernel_size=2, activation='relu',
    input_shape=(n_steps_in, n_features)
))
models.add(MaxPooling1D())
models.add(Flatten())
models.add(Dense(units=50, activation='relu'))
models.add(Dense(units=n_steps_out))
models.compile(optimizer='adam', loss='mse')
# fit model
models.fit(x=X, y=y, epochs=2000, verbose=False)

In [14]:
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
print(x_input)
print(x_input.shape)
yhat = models.predict(tf.cast(x_input, tf.float32), verbose=False)
print(yhat)
print(yhat.shape)

[[[70 75]
  [80 85]
  [90 95]]]
(1, 3, 2)
[[185.34235 207.36284]]
(1, 2)
